In [2]:
!unzip /content/drive/MyDrive/인공지능/경영빅데이터/data/ipo_원본데이터.zip -d /content/drive/MyDrive/인공지능/경영빅데이터/data

Archive:  /content/drive/MyDrive/인공지능/경영빅데이터/data/ipo_원본데이터.zip
  inflating: /content/drive/MyDrive/인공지능/경영빅데이터/data/ipo_원본데이터.csv  
